In [1]:
import re
import numpy as np
from sklearn import datasets
from sklearn.multiclass import OutputCodeClassifier
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.metrics import accuracy_score


def write_file(filename,\
                tr_error,\
                    test_error,\
                        b_s,\
                            support_vector_s,\
                                b_ve,\
                                support_vector_ve,\
                                    b_vi,\
                                    support_vector_vi,\
                                        ):
    with open(filename, 'w') as f:
        f.write('training error: ' + str(tr_error) + '\n')
        f.write('testing error: ' + str(test_error) + '\n')
        f.write('b_of_setosa: ' + str(b_s) + '\n')
        f.write('support_vector_indices_of_setosa: ' + str(support_vector_s) + '\n')
        f.write('b_of_versicolor: ' +str(b_ve) + '\n')
        f.write('support_vector_indices_of_versicolor: ' +str(support_vector_ve) + '\n')
        f.write('b_of_virginica: ' +str(b_vi) + '\n')
        f.write('support_vector_indices_of_virginica: ' +str(support_vector_vi))


class SVM(object):

    def __init__(self, training_dataset_, test_dataset_):
        self.training_dataset = training_dataset_
        self.test_dataset = test_dataset_
        self.classes = {}
        self.X_train = None
        self.Y_train = None
        self.X_test = None
        self.Y_test = None

        self.support_indecies = None
        self.train_errors = None
        self.val_errors = None
        self.train_loss = None
        self.val_loss = None
  
       
    def read_data(self):
        f = open(self.training_dataset, 'r')
        rows = list(re.split(' ', row) for row in re.split('\n', f.read())[:-1])
        names, self.Y_train = np.unique(list(row[-1] for row in rows), return_inverse=True)
        self.X_train = np.empty((0,4), float)
        f.close()
        for row in rows:
            self.X_train = np.append(self.X_train, np.array([np.array(row[:-1]).astype(float)]), axis = 0)
        f = open(self.test_dataset, 'r')
        f.close()
        
        f = open(self.test_dataset, 'r')
        rows = list(re.split(' ', row) for row in re.split('\n', f.read())[:-1])
        names, self.Y_test = np.unique(list(row[-1] for row in rows), return_inverse=True)
        self.X_test = np.empty((0,4), float)
        f.close()
        for row in rows:
            self.X_test = np.append(self.X_test, np.array([np.array(row[:-1]).astype(float)]), axis = 0)


    def SVM_general(self, C, kernel, degree, gamma, filename):
        Y_train_setosa = np.where(self.Y_train == 0, 1, -1)
        if gamma == 0: #gamma = 0 means that there's no gamma value
            clf_s = svm.SVC(C = C, kernel = kernel, degree = degree, decision_function_shape = 'ovo')
        else:
            clf_s = svm.SVC(C = C, kernel = kernel, degree = degree, gamma = gamma, decision_function_shape = 'ovo')
        clf_s.fit(self.X_train, Y_train_setosa) #fit the training data
        b_s = clf_s.intercept_ #retrieve the intercept bs
        b_s= ', '.join(str(i) for i in b_s) #string the bs up
        support_vector_s = clf_s.support_ #retrieve support vector
        support_vector_s = ', '.join(str(i) for i in support_vector_s) #wrap up the support vectors in string
        decision_function_s_train = clf_s.decision_function(self.X_train) #retrieve decision function
        decision_function_s_test = clf_s.decision_function(self.X_test)
        
        Y_train_ve = np.where(self.Y_train == 1, 1, -1)
        if gamma == 0:
            clf_ve = svm.SVC(C = C, kernel = kernel, degree = degree, decision_function_shape = 'ovo')
        else:
            clf_ve = svm.SVC(C = C, kernel = kernel, degree = degree, gamma = gamma, decision_function_shape = 'ovo')
        clf_ve.fit(self.X_train, Y_train_ve)
        b_ve = clf_ve.intercept_
        b_ve = ', '.join(str(i) for i in b_ve)
        support_vector_ve = clf_ve.support_
        support_vector_ve = ', '.join(str(i) for i in support_vector_ve)
        decision_function_ve_train = clf_ve.decision_function(self.X_train)
        decision_function_ve_test = clf_ve.decision_function(self.X_test)

        Y_train_vi = np.where(self.Y_train == 2, 1, -1)
        if gamma == 0:
            clf_vi = svm.SVC(C = C, kernel = kernel, degree = degree, decision_function_shape = 'ovo')
        else:
            clf_vi = svm.SVC(C = C, kernel = kernel, degree = degree, gamma = gamma, decision_function_shape = 'ovo')
        clf_vi.fit(self.X_train, Y_train_vi)
        b_vi = clf_vi.intercept_
        b_vi = ', '.join(str(i) for i in b_vi)
        support_vector_vi = clf_vi.support_
        support_vector_vi = ', '.join(str(i) for i in support_vector_vi)
        decision_function_vi_train = clf_vi.decision_function(self.X_train)
        decision_function_vi_test = clf_vi.decision_function(self.X_test)

        predict_class_tr = [] #to store the class of training vectors for prediction
        for i in range(len(decision_function_s_train)):
            deciList = [decision_function_s_train[i], decision_function_ve_train[i], \
                decision_function_vi_train[i]]
            predict_class_tr.append(deciList.index(max(deciList)))
        tr_error = 1 - accuracy_score(self.Y_train, predict_class_tr) #error = 1 - accuracy score

        predict_class_test = [] #to store the class of testing vectors for prediction
        for i in range(len(decision_function_s_test)):
            deciList = [decision_function_s_test[i], decision_function_ve_test[i], \
                decision_function_vi_test[i]]
            predict_class_test.append(deciList.index(max(deciList)))
        test_error = 1 - accuracy_score(self.Y_test, predict_class_test)
        
        write_file(filename,\
            tr_error,\
            test_error,\
                b_s,\
                    support_vector_s,\
                        b_ve,\
                        support_vector_ve,\
                            b_vi,\
                            support_vector_vi,\
                                )

        
    def SVM(self):
        
        #########################
        ## WRITE YOUR CODE HERE##
        #########################

        Y_train_setosa = np.where(self.Y_train == 0, 1, -1)
        clf_s = svm.SVC(C = 1e5, kernel = 'linear', decision_function_shape = 'ovo')
        clf_s.fit(self.X_train, Y_train_setosa)
        w_s = clf_s.coef_.flatten()
        w_s = ', '.join(str(i) for i in w_s)
        b_s = clf_s.intercept_
        b_s= ', '.join(str(i) for i in b_s)
        support_vector_s = clf_s.support_
        support_vector_s = ', '.join(str(i) for i in support_vector_s)
        decision_function_s_train = clf_s.decision_function(self.X_train)
        decision_function_s_test = clf_s.decision_function(self.X_test)

        Y_train_ve = np.where(self.Y_train == 1, 1, -1)
        clf_ve = svm.SVC(C = 1e5, kernel = 'linear', decision_function_shape = 'ovo')
        clf_ve.fit(self.X_train, Y_train_ve)
        w_ve = clf_ve.coef_.flatten()
        w_ve = ', '.join(str(i) for i in w_ve)
        b_ve = clf_ve.intercept_
        b_ve = ', '.join(str(i) for i in b_ve)
        support_vector_ve = clf_ve.support_
        support_vector_ve = ', '.join(str(i) for i in support_vector_ve)
        decision_function_ve_train = clf_ve.decision_function(self.X_train)
        decision_function_ve_test = clf_ve.decision_function(self.X_test)

        Y_train_vi = np.where(self.Y_train == 2, 1, -1)
        clf_vi = svm.SVC(C = 1e5, kernel = 'linear', decision_function_shape = 'ovo')
        clf_vi.fit(self.X_train, Y_train_vi)
        w_vi = clf_vi.coef_.flatten()
        w_vi = ', '.join(str(i) for i in w_vi)
        b_vi = clf_vi.intercept_
        b_vi = ', '.join(str(i) for i in b_vi)
        support_vector_vi = clf_vi.support_
        support_vector_vi = ', '.join(str(i) for i in support_vector_vi)
        decision_function_vi_train = clf_vi.decision_function(self.X_train)
        decision_function_vi_test = clf_vi.decision_function(self.X_test)

        predict_class_tr = []
        for i in range(len(decision_function_s_train)):
            deciList = [decision_function_s_train[i], decision_function_ve_train[i], \
                decision_function_vi_train[i]]
            predict_class_tr.append(deciList.index(max(deciList)))
        tr_error = 1 - accuracy_score(self.Y_train, predict_class_tr)

        predict_class_test = []
        for i in range(len(decision_function_s_test)):
            deciList = [decision_function_s_test[i], decision_function_ve_test[i], \
                decision_function_vi_test[i]]
            predict_class_test.append(deciList.index(max(deciList)))
        test_error = 1 - accuracy_score(self.Y_test, predict_class_test)

        with open("SVM_linear.txt", "w") as f:
            f.write('training error: ' + str(tr_error) + '\n')
            f.write('testing error: ' + str(test_error) + '\n')
            f.write('w_of_setosa: ' + str(w_s) + '\n')
            f.write('b_of_setosa: ' + str(b_s) + '\n')
            f.write('support_vector_indices_of_setosa: ' + str(support_vector_s) + '\n')
            f.write('w_of_versicolor: ' +str(w_ve) + '\n')
            f.write('b_of_versicolor: ' +str(b_ve) + '\n')
            f.write('support_vector_indices_of_versicolor: ' +str(support_vector_ve) + '\n')
            f.write('w_of_virginica: ' +str(w_vi) + '\n')
            f.write('b_of_virginica: ' +str(b_vi) + '\n')
            f.write('support_vector_indices_of_virginica: ' +str(support_vector_vi))

        # return train_loss, test_loss, support_vectors
    
    def SVM_slack(self, C):
    
        #########################
        ## WRITE YOUR CODE HERE##
        #########################  
          
        Y_train_setosa = np.where(self.Y_train == 0, 1, -1)
        clf_s = svm.SVC(C = C, kernel = 'linear', decision_function_shape = 'ovo')
        clf_s.fit(self.X_train, Y_train_setosa)
        w_s = clf_s.coef_.flatten()
        w_s = ', '.join(str(i) for i in w_s)
        b_s = clf_s.intercept_
        b_s= ', '.join(str(i) for i in b_s)
        support_vector_s = clf_s.support_
        support_vector_s = ', '.join(str(i) for i in support_vector_s)
        decision_function_s_train = clf_s.decision_function(self.X_train)
        decision_function_s_test = clf_s.decision_function(self.X_test)

        Y_train_ve = np.where(self.Y_train == 1, 1, -1)
        clf_ve = svm.SVC(C = C, kernel = 'linear', decision_function_shape = 'ovo')
        clf_ve.fit(self.X_train, Y_train_ve)
        w_ve = clf_ve.coef_.flatten()
        w_ve = ', '.join(str(i) for i in w_ve)
        b_ve = clf_ve.intercept_
        b_ve = ', '.join(str(i) for i in b_ve)
        support_vector_ve = clf_ve.support_
        support_vector_ve = ', '.join(str(i) for i in support_vector_ve)
        decision_function_ve_train = clf_ve.decision_function(self.X_train)
        decision_function_ve_test = clf_ve.decision_function(self.X_test)

        Y_train_vi = np.where(self.Y_train == 2, 1, -1)
        clf_vi = svm.SVC(C = C, kernel = 'linear', decision_function_shape = 'ovo')
        clf_vi.fit(self.X_train, Y_train_vi)
        w_vi = clf_vi.coef_.flatten()
        w_vi = ', '.join(str(i) for i in w_vi)
        b_vi = clf_vi.intercept_
        b_vi = ', '.join(str(i) for i in b_vi)
        support_vector_vi = clf_vi.support_
        support_vector_vi = ', '.join(str(i) for i in support_vector_vi)
        decision_function_vi_train = clf_vi.decision_function(self.X_train)
        decision_function_vi_test = clf_vi.decision_function(self.X_test)

        predict_class_tr = []
        for i in range(len(decision_function_s_train)):
            deciList = [decision_function_s_train[i], decision_function_ve_train[i], \
                decision_function_vi_train[i]]
            predict_class_tr.append(deciList.index(max(deciList)))
        tr_error = 1 - accuracy_score(self.Y_train, predict_class_tr)

        predict_class_test = []
        for i in range(len(decision_function_s_test)):
            deciList = [decision_function_s_test[i], decision_function_ve_test[i], \
                decision_function_vi_test[i]]
            predict_class_test.append(deciList.index(max(deciList)))
        test_error = 1 - accuracy_score(self.Y_test, predict_class_test)

        with open("SVM_slack_{}.txt".format(round(C,2)), "w") as f:
            f.write('training error: ' + str(tr_error) + '\n')
            f.write('testing error: ' + str(test_error) + '\n')
            f.write('w_of_setosa: ' + str(w_s) + '\n')
            f.write('b_of_setosa: ' + str(b_s) + '\n')
            f.write('support_vector_indices_of_setosa: ' + str(support_vector_s) + '\n')
            f.write('w_of_versicolor: ' +str(w_ve) + '\n')
            f.write('b_of_versicolor: ' +str(b_ve) + '\n')
            f.write('support_vector_indices_of_versicolor: ' +str(support_vector_ve) + '\n')
            f.write('w_of_virginica: ' +str(w_vi) + '\n')
            f.write('b_of_virginica: ' +str(b_vi) + '\n')
            f.write('support_vector_indices_of_virginica: ' +str(support_vector_vi))
        # return train_loss, test_loss, support_vectors
    
    def SVM_kernel_poly2(self, C):

        #########################
        ## WRITE YOUR CODE HERE##
        #########################    

        self.SVM_general(C, 'poly', 2, 0, "SVM_poly2.txt")


        # return train_loss, test_loss, support_vectors
    
    def SVM_kernel_poly3(self, C):
    
        #########################
        ## WRITE YOUR CODE HERE##
        #########################  

        self.SVM_general(C, 'poly', 3, 0, "SVM_poly3.txt")
        
        # return train_loss, test_loss, support_vectors
    
    def SVM_kernel_rbf(self, C):
    
        #########################
        ## WRITE YOUR CODE HERE##
        #########################  

        self.SVM_general(C, 'rbf', 3, 0.5, "SVM_rbf.txt")

        
        # return train_loss, test_loss, support_vectors
    
    def SVM_kernel_sigmoid(self, C):
    
        #########################
        ## WRITE YOUR CODE HERE##
        #########################    

        self.SVM_general(C, 'sigmoid', 3, 0.25, "SVM_sigmoid.txt")

    
        # return train_loss, test_loss, support_vectors

In [2]:
svm1 = SVM("train.txt", "test.txt")
svm1.read_data()
print(svm1.X_train)

svm1.SVM()

for i in range(1, 11):
    svm1.SVM_slack(0.1 * i)

svm1.SVM_kernel_poly2(1)
svm1.SVM_kernel_poly3(1)
svm1.SVM_kernel_rbf(1)
svm1.SVM_kernel_sigmoid(1)

#####################################
## Call different SVM with value C ##
#####################################

[[5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.1 1.5 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [5.  2.  3.5 1. ]
 [5.9 3.  4.2 1.5]
 [6.  2.2 4.  1. ]
 [6.1 2.9 4.7 1.4]
 [5.6 2.9 3.6 1.3]
 [6.7 3.1 4.4 1.4]
 [5.6 3.  4.5 1.5]
 [5.8 2.7 4.1 1. ]
 [6.2 2.2 4.5 1.5]
 [5.6 2.5 3.9 1.1]
 [5.9 3.2 4.8 1.8]
 [6.1 2.8 4.  1.3]
 [6.3 2.5 4.